In [1]:
import json
import re
import xml.etree.ElementTree as et
from xml.dom import minidom
import pandas as pd

In [7]:
with open("/Users/simon/Downloads/Telegram Desktop/ChatExport_2023-04-20 (1)/result.json") as f:
    txt = f.read()
    txtj = json.loads(txt)

In [8]:
# XML anlegen
corpus = et.Element("corpus")

# Listen für Dataframe anlegen
cols = ["id","date","from","reply_to","text"]
rows = []

for message in txtj["messages"]:
    
    #Reine Sytemmeldungen übergehen
    if message["type"] != "message":
        continue
    
    #Metadaten auslesen
    msg_id = str(message["id"])
    msg_date = str(message["date"])
    msg_from = message["from"]
    if msg_from == None:
        msg_from = "nA"
    if "reply_to_message_id" in message:
        msg_reply = str(message["reply_to_message_id"])
    else:
        msg_reply = "nA"
    
    #XML Element anlegen und Metadaten als Attribute schreiben
    message_element = et.SubElement(corpus, "message", {"id":msg_id,"date":msg_date,"from":msg_from, "reply_to":msg_reply})
    
    #Nachrichtentext auslesen
    msg_content = message["text"]
    
    #Nachrichten ohne Text übergehen
    if len(msg_content) == 0:
        continue
        
    #Falls die Nachrichten aus mehreren Textblöcken bestehen oder Links enthalten,
    #werden sie als Liste gefasst. Diese wird zu einem String ohne Zeilenumbrüche zusammengefügt.
    if type(msg_content) == list:
        txt_content = ""
        for part in msg_content:
            if type(part) == str:
                txt_content += part                
    elif type(msg_content) == str:
        txt_content = msg_content
                        
    msg_content = txt_content
    msg_content = msg_content.replace("\n", " ")
    msg_content = re.sub(r'\s+',' ',msg_content)
    
    #Text in XML schreiben
    message_element.text = msg_content
    
    # Listen befüllen
    rows.append({"id":msg_id,
                 "date":msg_date,
                 "from":msg_from,
                 "reply_to":msg_reply,
                 "text":msg_content})
    

In [19]:
xmlstr = minidom.parseString(et.tostring(corpus)).toprettyxml(indent="  ")
with open("telegram.xml", "w") as xmlout:
    xmlout.write(xmlstr)

In [16]:
# DataFrame anlegen und als Excel (oder CSV) speichern 
df = pd.DataFrame(rows, columns=cols)
df.to_excel("telegram.xlsx")

,id,date,from,reply_to,text
0,696194,2023-04-01T00:05:31,💣Horus💣,nA,Endlich ein 100er purmel 😍
1,696195,2023-04-01T00:07:13,Jenny 💐🍦,696194,Aus welcher Region? 🙈
2,696196,2023-04-01T00:07:41,💣Horus💣,nA,Leider Deutschland nicht regional 😫
3,696197,2023-04-01T00:10:00,Wolf,nA,niantic sollte endlich mal alternative input d...
4,696198,2023-04-01T00:12:03,Engelboy1989,696194,gw
...,...,...,...,...,...
1966,698738,2023-04-20T07:19:27,💣Horus💣,698735,Wenn man pokemon fängt 😁
1967,698739,2023-04-20T07:39:22,Vaka,nA,Bonbons ohne Ende aber kein Staub 😤
1968,698740,2023-04-20T07:47:07,Susa,698735,In dem man nicht alles pusht 😂
1969,698741,2023-04-20T07:51:14,💣Horus💣,698740,Genau 😁
